In [1]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os
from torch.nn import functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
import graduation_pytorch as gra

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchvision import transforms, datasets

## 1.增加随机旋转 3. 增加垂直翻转

In [2]:
# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_augs = transforms.Compose([
#         transforms.RandomRotation((-10,10)),
#         transforms.CenterCrop(size=224),
#         transforms.RandomHorizontalFlip(), #  随机的图像水平翻转 
#         transforms.RandomVerticalFlip(), # 垂直翻转
#         transforms.ToTensor(),
#         normalize
    
        transforms.Resize(size=256),
        transforms.RandomResizedCrop(size=224),
        transforms.RandomHorizontalFlip(), #  随机的图像水平翻转
        transforms.RandomVerticalFlip(), # 垂直翻转
        transforms.ToTensor(),
        normalize
    ])

test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])

In [3]:
train_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/train', transform=train_augs)
test_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/test', transform=test_augs)

In [4]:
pretrained_net = models.resnet34(pretrained=True)

In [5]:
# 所以我们应该将最后的fc成修改我们需要的输出类别数:  21
pretrained_net.fc = nn.Linear(512, 21)
print(pretrained_net.fc)
#此时，pretrained_net的fc层就被随机初始化了，但是其他层依然保存着预训练得到的参数。

Linear(in_features=512, out_features=21, bias=True)


In [6]:
# 实现池化类
class GeM(nn.Module):

    def __init__(self, p=2, eps=1e-6):
        super(GeM,self).__init__()
#         self.p = Parameter(torch.ones(1)*p)
        # 不设置为参数
        self.p = p
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)
        
    def __repr__(self):
        return self.__class__.__name__ + '()'
import math
# rmac 
def rmac(x, L=3, eps=1e-6):
    ovr = 0.4 # desired overlap of neighboring regions
    steps = torch.Tensor([2, 3, 4, 5, 6, 7]) # possible regions for the long dimension

    W = x.size(3)
    H = x.size(2)

    w = min(W, H)
    w2 = math.floor(w/2.0 - 1)

    b = (max(H, W)-w)/(steps-1)
    (tmp, idx) = torch.min(torch.abs(((w**2 - w*b)/w**2)-ovr), 0) # steps(idx) regions for long dimension

    # region overplus per dimension
    Wd = 0;
    Hd = 0;
    if H < W:  
        Wd = idx.item() + 1
    elif H > W:
        Hd = idx.item() + 1

    v = F.max_pool2d(x, (x.size(-2), x.size(-1)))
    v = v / (torch.norm(v, p=2, dim=1, keepdim=True) + eps).expand_as(v)

    for l in range(1, L+1):
        wl = math.floor(2*w/(l+1))
        wl2 = math.floor(wl/2 - 1)

        if l+Wd == 1:
            b = 0
        else:
            b = (W-wl)/(l+Wd-1)
        cenW = torch.floor(wl2 + torch.Tensor(range(l-1+Wd+1))*b) - wl2 # center coordinates
        if l+Hd == 1:
            b = 0
        else:
            b = (H-wl)/(l+Hd-1)
        cenH = torch.floor(wl2 + torch.Tensor(range(l-1+Hd+1))*b) - wl2 # center coordinates
            
        for i_ in cenH.tolist():
            for j_ in cenW.tolist():
                if wl == 0:
                    continue
                R = x[:,:,(int(i_)+torch.Tensor(range(wl)).long()).tolist(),:]
                R = R[:,:,:,(int(j_)+torch.Tensor(range(wl)).long()).tolist()]
                vt = F.max_pool2d(R, (R.size(-2), R.size(-1)))
                vt = vt / (torch.norm(vt, p=2, dim=1, keepdim=True) + eps).expand_as(vt)
                v += vt

    return v
# RMAC 类
class RMAC(nn.Module):

    def __init__(self, L=3, eps=1e-6):
        super(RMAC,self).__init__()
        self.L = L
        self.eps = eps

    def forward(self, x):
        return rmac(x, L=self.L, eps=self.eps)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'L=' + '{}'.format(self.L) + ')'

In [7]:
pretrained_net.avgpool = RMAC()

In [8]:
pretrained_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [9]:
output_params = list(map(id, pretrained_net.fc.parameters()))
feature_params = filter(lambda p: id(p) not in output_params, pretrained_net.parameters()) # 除fc层的参数

lr = 0.01
optimizer = optim.SGD([{'params': feature_params},
                       {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
                       lr=lr, weight_decay=0.001)

In [10]:
train_iter = DataLoader(train_dataset,
                            batch_size = 64, shuffle=True)
test_iter = DataLoader(test_dataset,
                            batch_size = 64, shuffle=False)

In [11]:
def train_fine_tuning(net, optimizer, batch_size=32, num_epochs=5):
    loss = torch.nn.CrossEntropyLoss()
    d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

In [12]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 13)

training on  cuda
epoch 1, loss 2.3552, train acc 0.388, test acc 0.693, time 36.1 sec
epoch 2, loss 0.6411, train acc 0.745, test acc 0.695, time 30.7 sec
epoch 3, loss 0.2595, train acc 0.858, test acc 0.733, time 30.1 sec
epoch 4, loss 0.1430, train acc 0.880, test acc 0.886, time 31.1 sec
epoch 5, loss 0.0855, train acc 0.908, test acc 0.902, time 31.0 sec
epoch 6, loss 0.0654, train acc 0.915, test acc 0.886, time 31.1 sec
epoch 7, loss 0.0447, train acc 0.933, test acc 0.874, time 31.7 sec
epoch 8, loss 0.0344, train acc 0.929, test acc 0.905, time 30.6 sec
epoch 9, loss 0.0286, train acc 0.936, test acc 0.917, time 32.2 sec
epoch 10, loss 0.0231, train acc 0.951, test acc 0.888, time 31.5 sec
epoch 11, loss 0.0201, train acc 0.949, test acc 0.919, time 31.5 sec
epoch 12, loss 0.0162, train acc 0.952, test acc 0.940, time 33.2 sec
epoch 13, loss 0.0139, train acc 0.954, test acc 0.926, time 33.1 sec


In [13]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/imgAug_avgp_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)

## 垂直，水平

In [12]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 2.3637, train acc 0.450, test acc 0.655, time 51.2 sec
epoch 2, loss 0.2709, train acc 0.836, test acc 0.655, time 28.6 sec
epoch 3, loss 0.1123, train acc 0.901, test acc 0.774, time 29.0 sec
epoch 4, loss 0.0723, train acc 0.917, test acc 0.817, time 29.2 sec
epoch 5, loss 0.0472, train acc 0.935, test acc 0.760, time 29.0 sec
epoch 6, loss 0.0343, train acc 0.933, test acc 0.802, time 28.9 sec
epoch 7, loss 0.0252, train acc 0.940, test acc 0.895, time 29.1 sec
epoch 8, loss 0.0212, train acc 0.951, test acc 0.857, time 28.9 sec
epoch 9, loss 0.0171, train acc 0.954, test acc 0.829, time 28.9 sec
epoch 10, loss 0.0169, train acc 0.945, test acc 0.802, time 29.1 sec
epoch 11, loss 0.0148, train acc 0.949, test acc 0.862, time 29.7 sec
epoch 12, loss 0.0099, train acc 0.967, test acc 0.900, time 29.4 sec
epoch 13, loss 0.0086, train acc 0.969, test acc 0.921, time 28.9 sec
epoch 14, loss 0.0072, train acc 0.971, test acc 0.902, time 29.4 sec
epoch 15, l

In [13]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 0.0979, train acc 0.971, test acc 0.902, time 28.8 sec
epoch 2, loss 0.0452, train acc 0.975, test acc 0.921, time 29.1 sec
epoch 3, loss 0.0267, train acc 0.977, test acc 0.921, time 29.5 sec
epoch 4, loss 0.0216, train acc 0.974, test acc 0.921, time 28.5 sec
epoch 5, loss 0.0229, train acc 0.966, test acc 0.893, time 28.9 sec


In [14]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 0.0959, train acc 0.975, test acc 0.898, time 28.8 sec
epoch 2, loss 0.0344, train acc 0.984, test acc 0.910, time 28.9 sec
epoch 3, loss 0.0301, train acc 0.977, test acc 0.905, time 29.1 sec
epoch 4, loss 0.0220, train acc 0.973, test acc 0.917, time 28.9 sec
epoch 5, loss 0.0193, train acc 0.975, test acc 0.869, time 29.2 sec


In [16]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 0.0830, train acc 0.978, test acc 0.862, time 29.2 sec
epoch 2, loss 0.0458, train acc 0.975, test acc 0.826, time 29.0 sec
epoch 3, loss 0.0202, train acc 0.979, test acc 0.914, time 28.9 sec
epoch 4, loss 0.0172, train acc 0.985, test acc 0.917, time 29.0 sec
epoch 5, loss 0.0143, train acc 0.978, test acc 0.912, time 29.7 sec


In [17]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/imgAug_avgp_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)

In [ ]:
## 旋转，中心裁剪，

In [33]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 2.1029, train acc 0.541, test acc 0.486, time 58.6 sec
epoch 2, loss 0.1632, train acc 0.914, test acc 0.864, time 48.9 sec
epoch 3, loss 0.0557, train acc 0.964, test acc 0.793, time 34.8 sec
epoch 4, loss 0.0318, train acc 0.968, test acc 0.848, time 28.8 sec
epoch 5, loss 0.0181, train acc 0.978, test acc 0.890, time 28.8 sec
epoch 6, loss 0.0108, train acc 0.988, test acc 0.888, time 29.3 sec
epoch 7, loss 0.0100, train acc 0.986, test acc 0.845, time 27.9 sec
epoch 8, loss 0.0072, train acc 0.986, test acc 0.890, time 28.2 sec
epoch 9, loss 0.0036, train acc 0.995, test acc 0.900, time 28.9 sec
epoch 10, loss 0.0046, train acc 0.994, test acc 0.852, time 29.2 sec
epoch 11, loss 0.0050, train acc 0.987, test acc 0.795, time 29.0 sec
epoch 12, loss 0.0029, train acc 0.990, test acc 0.874, time 28.8 sec
epoch 13, loss 0.0022, train acc 0.993, test acc 0.917, time 28.1 sec
epoch 14, loss 0.0016, train acc 0.998, test acc 0.924, time 29.0 sec
epoch 15, l

In [34]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 0.0169, train acc 0.997, test acc 0.917, time 28.6 sec
epoch 2, loss 0.0099, train acc 0.996, test acc 0.912, time 29.0 sec
epoch 3, loss 0.0060, train acc 0.998, test acc 0.890, time 28.8 sec
epoch 4, loss 0.0060, train acc 0.995, test acc 0.921, time 28.4 sec
epoch 5, loss 0.0026, train acc 0.999, test acc 0.926, time 28.8 sec


In [35]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/imgAug_avgp_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)

## 效果一般的图像增广

In [17]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 2.3810, train acc 0.385, test acc 0.431, time 61.1 sec
epoch 2, loss 0.5678, train acc 0.807, test acc 0.679, time 28.8 sec
epoch 3, loss 0.1963, train acc 0.912, test acc 0.824, time 29.0 sec
epoch 4, loss 0.0917, train acc 0.939, test acc 0.857, time 29.0 sec
epoch 5, loss 0.0545, train acc 0.948, test acc 0.869, time 29.0 sec
epoch 6, loss 0.0363, train acc 0.960, test acc 0.883, time 28.9 sec
epoch 7, loss 0.0241, train acc 0.970, test acc 0.888, time 29.3 sec
epoch 8, loss 0.0188, train acc 0.970, test acc 0.914, time 29.9 sec
epoch 9, loss 0.0141, train acc 0.976, test acc 0.917, time 28.9 sec
epoch 10, loss 0.0103, train acc 0.986, test acc 0.893, time 29.1 sec
epoch 11, loss 0.0084, train acc 0.979, test acc 0.917, time 28.9 sec
epoch 12, loss 0.0079, train acc 0.982, test acc 0.919, time 29.7 sec
epoch 13, loss 0.0055, train acc 0.989, test acc 0.936, time 29.2 sec
epoch 14, loss 0.0043, train acc 0.991, test acc 0.929, time 29.1 sec
epoch 15, l

In [18]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/imgAug_rmac_L=2_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)

##  avgpooling

In [22]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 2.8010, train acc 0.445, test acc 0.790, time 36.0 sec
epoch 2, loss 0.1846, train acc 0.890, test acc 0.802, time 28.9 sec
epoch 3, loss 0.0732, train acc 0.942, test acc 0.829, time 29.3 sec
epoch 4, loss 0.0387, train acc 0.960, test acc 0.895, time 28.5 sec
epoch 5, loss 0.0227, train acc 0.968, test acc 0.888, time 29.3 sec


In [23]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 10)

training on  cuda
epoch 1, loss 0.0952, train acc 0.974, test acc 0.886, time 28.8 sec
epoch 2, loss 0.0372, train acc 0.975, test acc 0.895, time 29.2 sec
epoch 3, loss 0.0206, train acc 0.985, test acc 0.886, time 28.7 sec
epoch 4, loss 0.0131, train acc 0.990, test acc 0.924, time 28.1 sec
epoch 5, loss 0.0096, train acc 0.986, test acc 0.929, time 29.1 sec
epoch 6, loss 0.0079, train acc 0.992, test acc 0.907, time 28.7 sec
epoch 7, loss 0.0092, train acc 0.981, test acc 0.845, time 29.9 sec
epoch 8, loss 0.0050, train acc 0.989, test acc 0.929, time 28.6 sec
epoch 9, loss 0.0029, train acc 0.995, test acc 0.931, time 29.1 sec
epoch 10, loss 0.0044, train acc 0.992, test acc 0.840, time 28.5 sec


In [24]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 1)

training on  cuda
epoch 1, loss 0.0326, train acc 0.993, test acc 0.924, time 28.8 sec


In [25]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/imgAug_avgp_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)